This notebook applies morphological smoothing and rule-based reclassification using external layers including DE Africas crop mask product.

### load packages

In [ ]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from odc.algo import xr_reproject
import matplotlib.pyplot as plt


/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### input files paths and set parameters

In [ ]:
output_crs='epsg:32735' # WGS84/UTM Zone 35S
# dict_map={'Trees':1,'Grassland':6,'Shrubland':8,'Cropland':10,'Vegetated Wetland':11,
#           'Open Water':12,'Settlements':13,'Irrigated Cropland':14} # class value dictionary
dict_map={'Forest':1,'Grassland':5,'Shrubland':7,'Perennial Cropland':9,'Annual Cropland':10,
          'Wetland':11,'Water Body':12,'Urban Settlement':13}
          
# file paths and attributes
# rwanda_shp='Data/Rwanda_Boundary.shp' # rwanda boundary shapefile
# rwanda_tiles_shp='Data/Rwanda_tiles_epsg32736_smaller.shp' # rwanda tiles shapefile
rwanda_tiles_shp='Results/Rwanda_random_sampling_AOIs.geojson' # rwanda tiles shapefile
river_network_shp='Data/hotosm_rwa_waterways_lines.shp' # OSM river network data
road_network_shp='Data/hotosm_rwa_roads_lines_filtered.shp' # OSM road network data
google_building_shp='Data/GoogleBuildingLayer_Rwanda.shp' # google bulding layer
# classification2021_raster='Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_stratified_samples_balanced_smaller_mosaic.tif' # land cover map of 2021
classification2021_raster='Results/Land_cover_prediction_Rwanda_2021_using_2015_sheme2_equal_samples_AOIs_mosaic.tif' # land cover map of 2021
hand_raster='Data/hand_Rwanda.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)
wsf2019_raster='Data/WSF2019_v1_Rwanda_clipped.tif' # 2019 WSF raster

## load layers

In [ ]:
# get bounding boxes of tiles covering rwanda
rwanda_tiles=gpd.read_file(rwanda_tiles_shp).to_crs(output_crs) 
tile_bboxes=rwanda_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021

# load external layers
# OSM road network
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) # import OSM road network data and reproject
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m
road_network_mask=xr_rasterize(gdf=road_network,da=landcover2021.squeeze(),
                               transform=landcover2021.geobox.transform,crs=output_crs) # # rasterise buffered OSM road network layer
# OSM river network
river_network=gpd.read_file(river_network_shp).to_crs(output_crs) # import OSM river network data and reproject
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute
river_network_mask=xr_rasterize(gdf=river_network,da=landcover2021.squeeze(),
                                transform=landcover2021.geobox.transform,crs=output_crs) # rasterise OSM river network layer
# import hand layer
hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze()

# import google bulding layer
google_buildings=gpd.read_file(google_building_shp).to_crs(output_crs) 
google_buildings=google_buildings.loc[google_buildings['confidence']>=0.6] # filter out low confidence polygons
google_buildings_mask=xr_rasterize(gdf=google_buildings,da=landcover2021.squeeze(),
                                  transform=landcover2021.geobox.transform,crs=output_crs)
wsf2019=xr.open_dataset(wsf2019_raster,engine="rasterio").astype(np.int32).squeeze() # import WSF2019 layers

### loop through tiles for reclassification and export as geotiffs

In [ ]:
for i in range(0,len(tile_bboxes)):
    # get tile bounding box
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    
    # load DE Africa crop mask 2019
    dc = datacube.Datacube(app='cropland_extent')
    query = {
        'time': ('2019'),
        'x': (x_min,x_max),
        'y': (y_min,y_max),
        'resolution':(-10, 10),
        'crs':output_crs,
        'output_crs': output_crs,
    }
    # now load the crop-mask using the query
    cm = dc.load(product='crop_mask',
                 **query).squeeze()
    ds_geobox=cm.geobox
    
    # clip land cover map 2021 to tile boundary
    landcover2021_tile=xr_reproject(landcover2021, ds_geobox, resampling="nearest") # clip to tile boundary
    np_landcover2021=landcover2021_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post=np_landcover2021.copy() # initialise post-processed numpy array
    
    # reclassify wetlands around (within 50m of) built-up areas as Forest
    urban_buffered=binary_dilation(np_landcover2021_post==dict_map['Urban Settlement'],footprint=disk(5)) # dilating built-up regions
    np_landcover2021_post[(urban_buffered==1)&(np_landcover2021_post==dict_map['Wetland'])]=dict_map['Forest'] # apply rule
    
    # assign pixels within crop mask but not classified as Perennial Cropland as Annual cropland
    np_crop_mask=cm['mask'].to_numpy()
    np_landcover2021_post[(np_landcover2021!=dict_map['Perennial Cropland'])&(np_crop_mask==1)]=dict_map['Annual Cropland']
    
    # mode filtering for a smoother classification map
#     np_landcover2021_post=modal(np_landcover2021_post,footprint=disk(2.5),mask=np_landcover2021!=0)
    np_landcover2021_post=modal(np_landcover2021_post,footprint=disk(2),mask=np_landcover2021_post!=0)
    
    # load hand layer
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand.to_array().squeeze().to_numpy()
    
    # Make sure water is (only occuring at bottom of watersheds) or fallen within OSM river networks
    river_network_mask_tile=xr_reproject(river_network_mask, ds_geobox, resampling="nearest")
    np_river_network_mask=river_network_mask_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post[((np_landcover2021_post==dict_map['Water Body'])&(np_hand<=45))|(np_river_network_mask==1)]=dict_map['Water Body'] # apply rules
    
    # assign pixels overlapping google building polygons or WSF 2019 as built-up
    google_buildings_mask_tile=xr_reproject(google_buildings_mask, ds_geobox, resampling="nearest")
    np_google_buildings_mask=google_buildings_mask_tile.squeeze().to_numpy() # data array to numpy array
    wsf2019_tile=xr_reproject(wsf2019, ds_geobox, resampling="nearest") # load and clip WSF layers
    np_wsf2019=wsf2019_tile.to_array().squeeze().to_numpy()
    np_landcover2021_post[(np_google_buildings_mask==1)|(np_wsf2019==255)]=dict_map['Urban Settlement'] # apply rules
    
    # assign pixesl overlapping buffered OSM road network as built-up class
    road_network_mask_tile=xr_reproject(road_network_mask, ds_geobox, resampling="nearest")
    np_road_network_mask=road_network_mask_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post[np_road_network_mask==1]=dict_map['Urban Settlement'] # burn in buffered OSM road network polygons
    
    # convert back result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,dims=['y','x'],coords={'y':landcover2021_tile.y.to_numpy(), 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(output_crs, inplace=True)
    
    # export as geotiff
    write_cog(landcover2021_tile_post, 'Results/Land_cover_prediction_postproc_Rwanda_2021_using_2015_sheme2_equal_random_samples_AOI_'+str(i)+'.tif', overwrite=True)

### mosaic all post-processed tiles

In [ ]:
! gdal_merge.py -o Results/Land_cover_prediction_postproc_Rwanda_2021_using_2015_sheme2_equal_random_samples_AOIs_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover_prediction_postproc_Rwanda_2021_using_2015_sheme2_equal_random_samples_AOI_*.tif